In [9]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [10]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [11]:
tf.keras.backend.clear_session()

In [12]:
subject = 'Minas Gerais - Consumo de Cimento (t)'
start_index = 60
split_index = 203 #Referente aos 230 anos de input  
train_split = split_index + 1 - 12*3

In [13]:
data = pd.read_csv('2003_mo_model_input_MG.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Unnamed: 0,Minas Gerais - IDH,Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Contratação Comercial (Bi R$),...,Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Minas Gerais - Consumo de Cimento (t)
0,2003-1,0.756537,0.334536,648.891160,8.242462,2.937752e+08,1.725077e+07,14.096054,2.759117e+08,5.020927e+07,...,8.202881e+10,0.724032,-5331.049150,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,247.134
1,2003-2,0.756708,0.335670,652.396314,8.236470,2.941480e+08,1.726959e+07,14.102603,2.760648e+08,5.018273e+07,...,8.222716e+10,0.690297,-5318.079644,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,305.708
2,2003-3,0.756879,0.336553,655.911573,8.230479,2.945208e+08,1.728841e+07,14.109152,2.762178e+08,5.016725e+07,...,8.234287e+10,0.669681,-5436.417870,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,284.168
3,2003-4,0.757050,0.336531,656.746430,8.224488,2.948935e+08,1.730723e+07,14.115701,2.763709e+08,5.016559e+07,...,8.235526e+10,0.660494,-5707.015274,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,286.594
4,2003-5,0.757222,0.336406,660.114672,8.218497,2.952663e+08,1.732605e+07,14.122250,2.765239e+08,5.011916e+07,...,8.245980e+10,0.648337,-5599.317941,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,312.455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,0.622619,1250.988075,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,633.858
236,2022-9,NaN,0.621866,1247.030458,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,612.428
237,2022-10,NaN,0.620866,1244.634022,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,579.831
238,2022-11,NaN,0.619585,1241.214294,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,559.950


In [14]:
input_data = data.iloc[:split_index + 1,1:-1]
# input_data = data.iloc[:split_index + 1,:]
input_data = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
input_data

,Minas Gerais - IDH,Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
0,-1.968698,-1.575817,-2.202855,-0.752424,-1.806910,-1.971018,-3.111816,-2.777792,1.874594,-1.361502,...,-1.063593,-0.822933,2.782450,0.097384,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489
1,-1.943192,-1.560482,-2.182122,-0.759292,-1.786501,-1.919572,-3.024730,-2.713279,1.869312,-1.359746,...,-1.063593,-0.818174,2.407943,0.100462,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073
2,-1.917685,-1.548550,-2.161328,-0.766160,-1.766091,-1.868126,-2.937643,-2.648766,1.866230,-1.358722,...,-1.063593,-0.815398,2.179073,0.072380,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505
3,-1.892178,-1.548848,-2.156390,-0.773028,-1.745682,-1.816680,-2.850557,-2.584252,1.865900,-1.358612,...,-1.063593,-0.815101,2.077086,0.008167,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220
4,-1.866672,-1.550544,-2.136466,-0.779896,-1.725272,-1.765234,-2.763471,-2.519739,1.856658,-1.352530,...,-1.063566,-0.812592,1.942128,0.033724,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1.081278,1.208028,1.070020,0.646276,0.920674,-0.608959,-0.023342,0.468241,-0.624727,1.575950,...,1.690001,0.667867,-0.689886,2.909383,-0.724085,-1.189161,3.148408,1.378950,0.117681,0.370628
200,1.064113,1.256499,1.058717,0.633557,0.902641,-0.600196,-0.045810,0.445578,-0.548742,1.764167,...,1.717926,0.640959,-0.441954,2.913017,-0.736434,-1.296499,3.219670,1.391539,0.092456,0.427775
201,1.046949,1.305916,1.081210,0.620837,0.884609,-0.591433,-0.068278,0.422914,-0.439805,1.909130,...,1.748758,0.615514,-0.132782,2.818435,-0.738433,-1.444029,3.421082,1.404128,0.067231,0.538287
202,1.029785,1.356297,1.085207,0.608118,0.866576,-0.582670,-0.090746,0.400250,-0.327203,2.040785,...,1.780444,0.593546,0.084061,2.905726,-0.738236,-1.444143,3.373840,1.416717,0.042006,0.666156


In [15]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      305.708
1      284.168
2      286.594
3      312.455
4      299.651
        ...   
235    612.428
236    579.831
237    559.950
238    559.950
239        NaN
Name: Minas Gerais - Consumo de Cimento (t), Length: 240, dtype: float64

In [16]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Minas Gerais - IDH,Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
60,-0.617077,-0.757795,-0.238292,-1.028493,-0.822444,0.927636,0.275859,-0.396951,-0.253335,0.485766,...,-0.907428,-1.023454,-0.537241,-0.615167,0.042292,0.488894,-0.717137,-0.601292,-0.315948,0.240813
61,-0.598696,-0.727075,-0.209865,-1.025680,-0.813752,0.973169,0.285191,-0.394134,-0.296485,0.526179,...,-0.899853,-1.024085,-0.479316,-0.525870,0.011943,-0.072763,-0.714911,-0.580436,-0.286095,0.246566
62,-0.580315,-0.690455,-0.153086,-1.022868,-0.805061,1.018702,0.294523,-0.391316,-0.338941,0.566133,...,-0.892039,-1.024307,-0.466312,-0.510476,-0.019339,-0.316940,-0.694168,-0.559580,-0.256243,0.315327
63,-0.561934,-0.651003,-0.114447,-1.020055,-0.796370,1.064234,0.303855,-0.388499,-0.380655,0.605591,...,-0.883981,-1.024104,-0.438778,-0.537853,-0.059264,0.180777,-0.700209,-0.538723,-0.226390,0.356189
64,-0.543553,-0.607147,-0.062262,-1.017242,-0.787679,1.109767,0.313188,-0.385682,-0.421583,0.644515,...,-0.875677,-1.023461,-0.457839,-0.446299,-0.104671,-0.212484,-0.660717,-0.517867,-0.196537,0.524165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1.334554,0.717612,-0.030629,1.239700,1.136971,-0.986304,0.021589,0.677638,-0.685481,-1.113174,...,1.272681,1.521503,-0.742853,0.615006,-0.619453,0.617009,-0.970725,0.980624,0.764972,-1.322806
164,1.337445,0.715300,-0.017187,1.195852,1.142243,-0.994112,0.041435,0.688796,-0.757697,-1.166440,...,1.288866,1.512161,-0.787367,0.546355,-0.611176,0.459632,-0.841670,0.991608,0.753473,-1.355523
165,1.340335,0.716462,-0.003286,1.152004,1.147515,-1.001919,0.061280,0.699954,-0.844729,-1.204659,...,1.304998,1.500026,-0.757940,0.421319,-0.599627,0.493659,-0.838453,1.002592,0.741973,-1.376175
166,1.343226,0.715461,0.000561,1.108156,1.152788,-1.009727,0.081125,0.711112,-0.914820,-1.250199,...,1.332157,1.486995,-0.917469,0.661467,-0.589237,0.617024,-1.006001,1.013577,0.730473,-1.414146


In [17]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

60     402.782000
61     429.874000
62     484.463000
63     498.487000
64     506.647000
          ...    
163    411.310457
164    406.131343
165    400.952229
166    395.773114
167    390.594000
Name: Minas Gerais - Consumo de Cimento (t), Length: 108, dtype: float64

In [18]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [19]:
reshaped_train, reshaped_target = create_batches(train_input, train_target, 36, 60)
reshaped_train.shape

(72, 36, 61)

In [20]:
# input para treinamento
test_input = input_data.iloc[train_split-36:split_index + 1]
test_input

,Minas Gerais - IDH,Minas Gerais - value,Minas Gerais - Produção de Cimento (t),Minas Gerais - Desemprego,Minas Gerais - PIB - Estadual,Minas Gerais - PIB - Construção Civil,Minas Gerais - PIB - Per Capita,Minas Gerais - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
132,0.558603,1.013808,0.681029,1.243600,0.846926,0.022034,0.277116,0.755622,1.160295,0.007546,...,0.467927,0.897908,1.590448,0.225519,-0.298056,-0.311347,-0.077688,0.623805,0.982447,0.041401
133,0.562216,1.008961,0.654027,1.276498,0.858253,-0.030478,0.243655,0.739933,1.253961,-0.126384,...,0.505071,0.970274,1.702213,0.413533,-0.281023,-0.313831,-0.030826,0.635788,0.979099,-0.019620
134,0.565829,1.009201,0.625159,1.309396,0.869581,-0.082991,0.210193,0.724244,1.424431,-0.206533,...,0.541297,1.040830,1.677438,0.431751,-0.265304,0.006739,-0.076111,0.647771,0.975750,0.029284
135,0.569442,1.006419,0.595518,1.342293,0.880908,-0.135503,0.176731,0.708555,1.530656,-0.271960,...,0.571731,1.107516,1.638599,0.414613,-0.251648,-0.486922,-0.205002,0.659754,0.972402,0.072516
136,0.573055,1.003323,0.565555,1.375191,0.892235,-0.188016,0.143269,0.692865,1.642943,-0.351172,...,0.611614,1.168977,1.691843,0.566348,-0.231997,-0.559925,-0.132229,0.671737,0.969054,-0.064532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1.081278,1.208028,1.070020,0.646276,0.920674,-0.608959,-0.023342,0.468241,-0.624727,1.575950,...,1.690001,0.667867,-0.689886,2.909383,-0.724085,-1.189161,3.148408,1.378950,0.117681,0.370628
200,1.064113,1.256499,1.058717,0.633557,0.902641,-0.600196,-0.045810,0.445578,-0.548742,1.764167,...,1.717926,0.640959,-0.441954,2.913017,-0.736434,-1.296499,3.219670,1.391539,0.092456,0.427775
201,1.046949,1.305916,1.081210,0.620837,0.884609,-0.591433,-0.068278,0.422914,-0.439805,1.909130,...,1.748758,0.615514,-0.132782,2.818435,-0.738433,-1.444029,3.421082,1.404128,0.067231,0.538287
202,1.029785,1.356297,1.085207,0.608118,0.866576,-0.582670,-0.090746,0.400250,-0.327203,2.040785,...,1.780444,0.593546,0.084061,2.905726,-0.738236,-1.444143,3.373840,1.416717,0.042006,0.666156


In [21]:
reshaped_test, reshaped_test_target = create_batches(test_input, target_data, 36, 132)
reshaped_test.shape 

(36, 36, 61)

In [22]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * 6 + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [23]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 10)
    target,target_val = validation_splitter(train_target, 10)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [24]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [43]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[4044912764, 471336047, 248871671, 2571038028, 2157424748, 1030025407, 2913884917, 444418238, 4286526287, 3647672402, 3125054139, 4271766002, 4186366032, 3153313978, 109343731, 3435447617, 1589507121, 1743928685, 2333774836, 4279528964, 1415869528, 3495430536, 196986957, 3346204416, 1419690354]


Step: 0 ___________________________________________
val_loss: 412.9736633300781
winner_seed: 4044912764


Step: 1 ___________________________________________
val_loss: 2854.00244140625


Step: 2 ___________________________________________
val_loss: 3977.924072265625


Step: 3 ___________________________________________
val_loss: 164.25062561035156
winner_seed: 2571038028


Step: 4 ___________________________________________
val_loss: 119.44486236572266
winner_seed: 2157424748


Step: 5 ___________________________________________
val_loss: 2107.956298828125


Step: 6 ___________________________________________
val_loss: 557.0610961914062


Step: 7 ___________________________________________
val

In [26]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=4279528964)

Epoch 1/10000
3/3 [==============================] - 2s 171ms/step - loss: 294218.0625 - val_loss: 33302.1680
Epoch 2/10000
3/3 [==============================] - 0s 10ms/step - loss: 12157.5098 - val_loss: 12490.7080
Epoch 3/10000
3/3 [==============================] - 0s 10ms/step - loss: 7641.0674 - val_loss: 49301.1875
Epoch 4/10000
3/3 [==============================] - 0s 10ms/step - loss: 14928.2676 - val_loss: 14556.6973
Epoch 5/10000
3/3 [==============================] - 0s 10ms/step - loss: 16399.9082 - val_loss: 22847.3125
Epoch 6/10000
3/3 [==============================] - 0s 10ms/step - loss: 8158.8950 - val_loss: 11187.3887
Epoch 7/10000
3/3 [==============================] - 0s 10ms/step - loss: 6590.9443 - val_loss: 6030.6919
Epoch 8/10000
3/3 [==============================] - 0s 10ms/step - loss: 6840.5625 - val_loss: 9747.8271
Epoch 9/10000
3/3 [==============================] - 0s 10ms/step - loss: 16369.0098 - val_loss: 9512.4395
Epoch 10/10000
3/3 [=============

Epoch 77/10000
3/3 [==============================] - 0s 10ms/step - loss: 12981.7109 - val_loss: 7037.2300
Epoch 78/10000
3/3 [==============================] - 0s 10ms/step - loss: 10341.9951 - val_loss: 12205.6738
Epoch 79/10000
3/3 [==============================] - 0s 10ms/step - loss: 16175.4043 - val_loss: 5462.7007
Epoch 80/10000
3/3 [==============================] - 0s 10ms/step - loss: 8131.4414 - val_loss: 9100.2090
Epoch 81/10000
3/3 [==============================] - 0s 9ms/step - loss: 13857.8867 - val_loss: 42920.9375
Epoch 82/10000
3/3 [==============================] - 0s 10ms/step - loss: 15388.3457 - val_loss: 23257.5371
Epoch 83/10000
3/3 [==============================] - 0s 10ms/step - loss: 8620.7178 - val_loss: 9457.7119
Epoch 84/10000
3/3 [==============================] - 0s 10ms/step - loss: 6503.9229 - val_loss: 15983.6592
Epoch 85/10000
3/3 [==============================] - 0s 10ms/step - loss: 7225.1289 - val_loss: 11105.4277
Epoch 86/10000
3/3 [========

3/3 [==============================] - 0s 10ms/step - loss: 5121.9331 - val_loss: 4758.4341
Epoch 228/10000
3/3 [==============================] - 0s 10ms/step - loss: 4506.8784 - val_loss: 7118.9658
Epoch 229/10000
3/3 [==============================] - 0s 10ms/step - loss: 4469.8281 - val_loss: 4851.4624
Epoch 230/10000
3/3 [==============================] - 0s 10ms/step - loss: 4688.1250 - val_loss: 6876.5029
Epoch 231/10000
3/3 [==============================] - 0s 10ms/step - loss: 4480.4707 - val_loss: 13484.0645
Epoch 232/10000
3/3 [==============================] - 0s 10ms/step - loss: 6196.8379 - val_loss: 5813.3833
Epoch 233/10000
3/3 [==============================] - 0s 10ms/step - loss: 4444.0542 - val_loss: 19409.0176
Epoch 234/10000
3/3 [==============================] - 0s 10ms/step - loss: 6846.9507 - val_loss: 4629.8438
Epoch 235/10000
3/3 [==============================] - 0s 10ms/step - loss: 4319.1997 - val_loss: 4656.1016
Epoch 236/10000
3/3 [=====================

3/3 [==============================] - 0s 10ms/step - loss: 10338.0586 - val_loss: 6032.9775
Epoch 378/10000
3/3 [==============================] - 0s 10ms/step - loss: 9356.4785 - val_loss: 6016.0244
Epoch 379/10000
3/3 [==============================] - 0s 10ms/step - loss: 9560.7881 - val_loss: 5542.8193
Epoch 380/10000
3/3 [==============================] - 0s 10ms/step - loss: 9187.0146 - val_loss: 8338.0273
Epoch 381/10000
3/3 [==============================] - 0s 10ms/step - loss: 11246.2354 - val_loss: 29346.4648
Epoch 382/10000
3/3 [==============================] - 0s 10ms/step - loss: 11365.7803 - val_loss: 6877.9565
Epoch 383/10000
3/3 [==============================] - 0s 10ms/step - loss: 6516.0420 - val_loss: 5465.4800
Epoch 384/10000
3/3 [==============================] - 0s 10ms/step - loss: 8373.2207 - val_loss: 6371.5068
Epoch 385/10000
3/3 [==============================] - 0s 10ms/step - loss: 6552.6509 - val_loss: 24849.6504
Epoch 386/10000
3/3 [==================

Epoch 528/10000
3/3 [==============================] - 0s 9ms/step - loss: 8406.8594 - val_loss: 12926.2666
Epoch 529/10000
3/3 [==============================] - 0s 9ms/step - loss: 6260.4473 - val_loss: 10258.5391
Epoch 530/10000
3/3 [==============================] - 0s 9ms/step - loss: 12021.7500 - val_loss: 5286.0103
Epoch 531/10000
3/3 [==============================] - 0s 9ms/step - loss: 6022.1206 - val_loss: 19007.6523
Epoch 532/10000
3/3 [==============================] - 0s 9ms/step - loss: 7796.1328 - val_loss: 5628.2422
Epoch 533/10000
3/3 [==============================] - 0s 9ms/step - loss: 5887.8994 - val_loss: 10847.9600
Epoch 534/10000
3/3 [==============================] - 0s 9ms/step - loss: 13514.6387 - val_loss: 34254.6367
Epoch 535/10000
3/3 [==============================] - 0s 9ms/step - loss: 13188.6660 - val_loss: 5980.9302
Epoch 536/10000
3/3 [==============================] - 0s 10ms/step - loss: 5682.5518 - val_loss: 11225.5645
Epoch 537/10000
3/3 [======

...

3/3 [==============================] - 0s 9ms/step - loss: 1965.7299 - val_loss: 237.9342
Epoch 1897/10000
3/3 [==============================] - 0s 9ms/step - loss: 2035.1188 - val_loss: 430.8613
Epoch 1898/10000
3/3 [==============================] - 0s 9ms/step - loss: 2264.8496 - val_loss: 1258.7422
Epoch 1899/10000
3/3 [==============================] - 0s 9ms/step - loss: 2925.0984 - val_loss: 1334.1599
Epoch 1900/10000
3/3 [==============================] - 0s 9ms/step - loss: 2047.3239 - val_loss: 389.3413
Epoch 1901/10000
3/3 [==============================] - 0s 9ms/step - loss: 1774.7506 - val_loss: 521.9417
Epoch 1902/10000
3/3 [==============================] - 0s 9ms/step - loss: 2001.1687 - val_loss: 2870.2869
Epoch 1903/10000
3/3 [==============================] - 0s 9ms/step - loss: 3438.8206 - val_loss: 425.4360
Epoch 1904/10000
3/3 [==============================] - 0s 9ms/step - loss: 1913.9655 - val_loss: 491.6700
Epoch 1905/10000
3/3 [=============================

3/3 [==============================] - 0s 9ms/step - loss: 4795.4551 - val_loss: 1613.2897
Epoch 2049/10000
3/3 [==============================] - 0s 9ms/step - loss: 2870.4402 - val_loss: 1107.8821
Epoch 2050/10000
3/3 [==============================] - 0s 9ms/step - loss: 2322.2407 - val_loss: 1013.3195
Epoch 2051/10000
3/3 [==============================] - 0s 9ms/step - loss: 2278.9827 - val_loss: 939.7161
Epoch 2052/10000
3/3 [==============================] - 0s 9ms/step - loss: 2401.2646 - val_loss: 711.0287
Epoch 2053/10000
3/3 [==============================] - 0s 9ms/step - loss: 2236.4084 - val_loss: 566.6633
Epoch 2054/10000
3/3 [==============================] - 0s 9ms/step - loss: 2100.5703 - val_loss: 2733.8650
Epoch 2055/10000
3/3 [==============================] - 0s 9ms/step - loss: 3429.4934 - val_loss: 2141.6819
Epoch 2056/10000
3/3 [==============================] - 0s 9ms/step - loss: 3344.1365 - val_loss: 594.2831
Epoch 2057/10000
3/3 [===========================

3/3 [==============================] - 0s 9ms/step - loss: 2673.2639 - val_loss: 1013.3701
Epoch 2201/10000
3/3 [==============================] - 0s 9ms/step - loss: 2338.1421 - val_loss: 2681.6499
Epoch 2202/10000
3/3 [==============================] - 0s 9ms/step - loss: 4871.7031 - val_loss: 5092.7212
Epoch 2203/10000
3/3 [==============================] - 0s 9ms/step - loss: 4000.1584 - val_loss: 2023.7585
Epoch 2204/10000
3/3 [==============================] - 0s 9ms/step - loss: 2710.9309 - val_loss: 1418.8086
Epoch 2205/10000
3/3 [==============================] - 0s 9ms/step - loss: 2328.0686 - val_loss: 1461.5117
Epoch 2206/10000
3/3 [==============================] - 0s 9ms/step - loss: 2284.7249 - val_loss: 1552.0715
Epoch 2207/10000
3/3 [==============================] - 0s 9ms/step - loss: 2462.5583 - val_loss: 2594.9622
Epoch 2208/10000
3/3 [==============================] - 0s 9ms/step - loss: 2767.3206 - val_loss: 1505.8340
Epoch 2209/10000
3/3 [=======================

3/3 [==============================] - 0s 9ms/step - loss: 2106.3491 - val_loss: 2389.3093
Epoch 2353/10000
3/3 [==============================] - 0s 9ms/step - loss: 2121.0784 - val_loss: 2572.8416
Epoch 2354/10000
3/3 [==============================] - 0s 9ms/step - loss: 2199.7488 - val_loss: 1171.3971
Epoch 2355/10000
3/3 [==============================] - 0s 9ms/step - loss: 2010.1838 - val_loss: 921.0113
Epoch 2356/10000
3/3 [==============================] - 0s 9ms/step - loss: 2085.5657 - val_loss: 562.3207
Epoch 2357/10000
3/3 [==============================] - 0s 9ms/step - loss: 2035.4137 - val_loss: 773.9413
Epoch 2358/10000
3/3 [==============================] - 0s 9ms/step - loss: 2094.3088 - val_loss: 1620.0564
Epoch 2359/10000
3/3 [==============================] - 0s 9ms/step - loss: 3103.8979 - val_loss: 5629.5615
Epoch 2360/10000
3/3 [==============================] - 0s 9ms/step - loss: 3419.2161 - val_loss: 897.0342
Epoch 2361/10000
3/3 [===========================

3/3 [==============================] - 0s 9ms/step - loss: 2073.2859 - val_loss: 1633.9637
Epoch 2429/10000
3/3 [==============================] - 0s 9ms/step - loss: 1804.7728 - val_loss: 1607.9619
Epoch 2430/10000
3/3 [==============================] - 0s 9ms/step - loss: 1949.5499 - val_loss: 1992.9733
Epoch 2431/10000
3/3 [==============================] - 0s 9ms/step - loss: 1900.6190 - val_loss: 6551.9839
Epoch 2432/10000
3/3 [==============================] - 0s 9ms/step - loss: 3724.7610 - val_loss: 1632.2861
Epoch 2433/10000
3/3 [==============================] - 0s 9ms/step - loss: 1878.6265 - val_loss: 2344.0481
Epoch 2434/10000
3/3 [==============================] - 0s 9ms/step - loss: 1773.9753 - val_loss: 2932.7168
Epoch 2435/10000
3/3 [==============================] - 0s 9ms/step - loss: 2471.5813 - val_loss: 1469.3796
Epoch 2436/10000
3/3 [==============================] - 0s 9ms/step - loss: 2854.4973 - val_loss: 1361.4456
Epoch 2437/10000
3/3 [=======================

In [27]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    for i in range(len(test_target)):
        input_test = test_input[i:i+1]
        prediction = model.predict(input_test)
        error = np.abs(prediction - test_target[start_target + i])
        errors.append(error)
        error_percent.append(error / test_target[start_target + i])
        print(f"Month-{i + 1} - Error: {error}")
    
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [28]:
errors, mae, mape = mae_mape_calculator(trained_model, 
                                        reshaped_test, 
                                        reshaped_test_target, 0)

1/1 [==============================] - 0s 299ms/step
Month-1 - Error: [[48.49591]]
1/1 [==============================] - 0s 14ms/step
Month-2 - Error: [[33.17685]]
1/1 [==============================] - 0s 13ms/step
Month-3 - Error: [[0.85827637]]
1/1 [==============================] - 0s 13ms/step
Month-4 - Error: [[4.110077]]
1/1 [==============================] - 0s 13ms/step
Month-5 - Error: [[21.67804]]
1/1 [==============================] - 0s 13ms/step
Month-6 - Error: [[23.412079]]
1/1 [==============================] - 0s 13ms/step
Month-7 - Error: [[66.674835]]
1/1 [==============================] - 0s 13ms/step
Month-8 - Error: [[43.75461]]
1/1 [==============================] - 0s 13ms/step
Month-9 - Error: [[10.985504]]
1/1 [==============================] - 0s 13ms/step
Month-10 - Error: [[18.532166]]
1/1 [==============================] - 0s 13ms/step
Month-11 - Error: [[94.400116]]
1/1 [==============================] - 0s 13ms/step
Month-12 - Error: [[1.6567383]]
1/1 

In [29]:
display(mae)
display(mape)

65.86651

0.13666737

In [30]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}")
            target_sum = 0
            prediction_sum = 0
            
        input_test = test_input[i:i+1]
        prediction = model.predict(input_test)
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [31]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 0)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[4918.8496]] - Target[4938.444]| =  Error: [[19.594238]]
1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[5193.7417]] - Target[5580.459999999999]| =  Error: [[386.71826]]
1/1 [==============================] - 0s 13ms/step
Ano-11: |Prediction[[5213.5815]] - Target[6223.561000000001]| =  Error: [[1009.9795]]


[array([[19.594238]], dtype=float32),
 array([[386.71826]], dtype=float32),
 array([[1009.9795]], dtype=float32)]

472.09732

0.07851651